In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=b00211891e548d324020054a3ad38199b394ee837ff97f0d718a3bcb264454ad
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F16 '''
def F16(X):
    f = bn.F16()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.12 ,random_state=0 , l1_ratio=0.81, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_16_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_16_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(20, 30):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-20,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 20
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=417864, Sun Apr 19 13:23:13 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 -1.801812808110249e+02 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
termination on tolfun=1e-11 (Sun Apr 19 13:23:13 2020)
final/bestever f-value = -1.801813e+02 -1.801813e+02
incumbent solution: [ 3.00323157 -3.83340886  0.66294705  3.74100324 -3.1812557  -1.69308497
  4.22450056  3.75699704 ...]
std deviations: [2.43903838 2.43911702 2.43893329 2.43883869 2.43878123 2.43918076
 2.43907428 2.43876109 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=378521, Sun Apr 19 13:23:13 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 -1.801146458141778e+02 1.0e+00 2.44e+00  2e+00  2e+00 0:00.0
termination on tolfun=1e-11 (Sun Apr 19 13:23:13 2020)
final/bestever f-value = -1.801146e+02 -1.801146e+02
incumbent solution: [ 2.60682512 -4.05565702  0.79747175  4.990

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.18128081] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.11464581] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.18144254] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in functi

    2     38 -1.802070193061392e+02 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 -1.802077412848565e+02 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.18893533] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   35    665 -1.802077409687834e+02 1.1e+00 1.96e+00  2e+00  2e+00 0:03.4
   65   1235 -1.802077784988801e+02 1.2e+00 1.98e+00  2e+00  2e+00 0:07.4
   97   1843 -1.802077765673175e+02 1.2e+00 1.97e+00  2e+00  2e+00 0:12.5
  100   1900 -1.802077781820812e+02 1.2e+00 1.98e+00  2e+00  2e+00 0:12.8
  174   3306 -1.802076585433443e+02 1.4e+00 1.92e+00  2e+00  2e+00 0:19.8
  200   3800 -1.802075052476108e+02 1.4e+00 1.84e+00  2e+00  2e+00 0:22.4
  294   5586 -1.802076984091065e+02 1.5e+00 1.83e+00  2e+00  2e+00 0:31.4
  300   5700 -1.802077161386638e+02 1.5e+00 1.83e+00  2e+00  2e+00 0:32.0
  400   7600 -1.802076972709547e+02 1.6e+00 1.90e+00  2e+00  2e+00 0:41.6
  500   9500 -1.802077796297301e+02 1.6e+00 1.70e+00  1e+00  2e+00 0:51.2
  600  11400 -1.802077412850886e+02 1.7e+00 1.48e+00  1e+00  2e+00 1:00.8
  700  13300 -1.802077228216896e+02 1.8e+00 1.77e+00  1e+00  2e+00 1:10.4
  800  15200 -1.802075673900838e+02 1.9e+00 2.06e+00  2e+00  2e+00 1:20.0
  900  17100 -1.802077804856278e+02 1.

...

    3     57 -1.801902321335964e+02 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   35    665 -1.802077806223350e+02 1.1e+00 1.90e+00  2e+00  2e+00 0:03.4
   78   1482 -1.802069921474326e+02 1.2e+00 1.85e+00  2e+00  2e+00 0:07.4
  100   1900 -1.802074598499332e+02 1.3e+00 1.77e+00  2e+00  2e+00 0:09.5
  163   3097 -1.802077686399466e+02 1.3e+00 1.62e+00  2e+00  2e+00 0:15.5
  200   3800 -1.802072633594606e+02 1.4e+00 1.56e+00  1e+00  2e+00 0:19.0
  285   5415 -1.802077763711498e+02 1.4e+00 1.49e+00  1e+00  2e+00 0:27.1
  300   5700 -1.802077799688001e+02 1.4e+00 1.47e+00  1e+00  2e+00 0:28.5
  400   7600 -1.802076934792414e+02 1.5e+00 1.44e+00  1e+00  1e+00 0:38.0
  500   9500 -1.802077341525863e+02 1.5e+00 1.70e+00  1e+00  2e+00 0:47.4
  600  11400 -1.802077735681615e+02 1.7e+00 1.82e+00  2e+00  2e+00 0:56.9
  700  13300 -1.802077760803457e+02 1.8e+00 1.84e+00  2e+00  2e+00 1:06.4
  800  15200 -1.802077801506677e+02 1.8e+00 1.90e+00  1e+00  2e+00 1:16.0
  900  17100 -1.802074223439635e+02 1.

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.18128081] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.11464581] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


termination on tolfun=1e-11 (Sun Apr 19 15:59:40 2020)
final/bestever f-value = -1.801146e+02 -1.801146e+02
incumbent solution: [-4.65862006 -4.41633365 -4.54895068  2.68908077  2.91398892  3.92330862
  0.08825342 -0.05430512 ...]
std deviations: [2.43667971 2.43702182 2.43709467 2.43658534 2.43684817 2.43698954
 2.43661454 2.43739981 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=390116, Sun Apr 19 15:59:40 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 -1.802074703246253e+02 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 -1.802077676190176e+02 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.15478741] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.2044397] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 -1.802069745778631e+02 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   35    665 -1.802070673440310e+02 1.1e+00 1.93e+00  2e+00  2e+00 0:03.4
   77   1463 -1.802077718246833e+02 1.2e+00 1.91e+00  2e+00  2e+00 0:07.4
  100   1900 -1.802077430624705e+02 1.3e+00 1.90e+00  2e+00  2e+00 0:09.6
  163   3097 -1.802077517200212e+02 1.4e+00 1.80e+00  2e+00  2e+00 0:15.6
  200   3800 -1.802077803596305e+02 1.4e+00 1.76e+00  2e+00  2e+00 0:19.2
  284   5396 -1.802077744145857e+02 1.5e+00 1.71e+00  2e+00  2e+00 0:27.2
  300   5700 -1.802077240465999e+02 1.5e+00 1.82e+00  2e+00  2e+00 0:28.7
  400   7600 -1.802077424816286e+02 1.6e+00 2.00e+00  2e+00  2e+00 0:38.3
  500   9500 -1.802077804424029e+02 1.7e+00 1.89e+00  2e+00  2e+00 0:47.9
  600  11400 -1.802077806222900e+02 1.7e+00 1.98e+00  2e+00  2e+00 0:57.5
  700  13300 -1.802077343294150e+02 1.8e+00 2.03e+00  2e+00  2e+00 1:07.2
  800  15200 -1.802077704872565e+02 1.9e+00 1.95e+00  2e+00  2e+00 1:16.8
  900  17100 -1.802077774155933e+02 2.

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('sample_data\\F16_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F16(X_Values.iloc[i,:200])
    SVM_Fun [i] = F16(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F16(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([-195.92060499]), count=array([1])) 10.289585471148902
SVM
ModeResult(mode=array([-187.83217662]), count=array([1])) 4.323150783813076
ELN
ModeResult(mode=array([-186.61488978]), count=array([1])) 4.4068578648880425
